
## 🎯 Practice Exercises
## Exercise 1: Build Your First Stateful Agent

### Task
Create an agent with three custom tools:
1. **Weather tool:** Returns simulated weather for a given city
2. **Dictionary tool:** Looks up word definitions (simulate with a small dict)
3. **Web search tool:** Uses DuckDuckGo to search the web for information

### Requirements
1. Define tools using `@tool` decorator
2. Bind tools to LLM
3. Implement conditional routing (agent decides which tool to use)
4. Handle cases where no tool is needed
5. Install DuckDuckGo search: `pip install duckduckgo-search`
6. Use `DDGS().text()` method for web searches

### Example Queries
- "What's the weather in Lagos?" → Uses weather tool
- "Define the word 'ephemeral'" → Uses dictionary
- "Search for latest AI news" → Uses DuckDuckGo web search
- "What's the capital of France?" → No tool needed


In [22]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
from duckduckgo_search import DDGS
import random
import os

print("✅ All imports successful")

✅ All imports successful


In [23]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [24]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Lower temperature for more precise tool usage
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [25]:
@tool
def get_weather(city: str) -> str:
    """
    Returns simulated weather information for a given city in Nigeria.
    """
    city = city.strip().title()

    simulated_weather_db = {
        "Lagos": {"temp": 30, "condition": "Humid and cloudy"},
        "Abuja": {"temp": 28, "condition": "Sunny"},
        "Ibadan": {"temp": 29, "condition": "Partly cloudy"},
        "Abeokuta": {"temp": 31, "condition": "Hot and sunny"},
        "Ilorin": {"temp": 32, "condition": "Dry and sunny"},
        "Benin": {"temp": 30, "condition": "Cloudy"},
        "Enugu": {"temp": 27, "condition": "Mild with light rain"},
        "Port Harcourt": {"temp": 29, "condition": "Rainy and humid"},
    }

    if city not in simulated_weather_db:
        temp = random.randint(26, 34)
        condition = random.choice(
            ["Sunny", "Cloudy", "Partly cloudy", "Light rain", "Humid"]
        )
    else:
        temp = simulated_weather_db[city]["temp"]
        condition = simulated_weather_db[city]["condition"]

    return (
        f"Weather in {city}, Nigeria:\n"
        f"Temperature: {temp}°C\n"
        f"Condition: {condition}"
    )
print("☁️Weather tool created")

☁️Weather tool created


In [26]:
@tool
def dictionary_lookup(word: str) -> str:
    """
    Looks up the definition of a word using a simulated dictionary.
    """
    word = word.strip().lower()

    simulated_dictionary = {
        "voltage": "The electrical potential difference between two points.",
        "current": "The flow of electric charge through a conductor.",
        "resistance": "The opposition to the flow of electric current.",
        "transformer": "An electrical device used to increase or decrease voltage levels.",
        "frequency": "The number of cycles of a repeating signal per second, measured in hertz.",
        "algorithm": "A step-by-step procedure for solving a problem or performing a computation.",
        "neural network": "A computational model inspired by the human brain, used in machine learning.",
    }

    if word in simulated_dictionary:
        return f"{word.title()}:\n{simulated_dictionary[word]}"
    else:
        return (
            f"No definition found for '{word}'. "
            "This is a simulated dictionary with limited entries."
        )
print("📔Dictionary tool created")

📔Dictionary tool created


In [27]:
@tool
def web_search(query: str) -> str:
    """
    Searches the web using DuckDuckGo and returns summarized results.
    """
    query = query.strip()

    results = []

    with DDGS() as ddgs:
        search_results = ddgs.text(query, max_results=5)

        for r in search_results:
            results.append(
                f"- {r.get('title')}\n  {r.get('body')}\n  Source: {r.get('href')}"
            )

    if not results:
        return "No results found."

    return "Web search results:\n\n" + "\n\n".join(results)


In [28]:
# Creating list of tools
tools = [get_weather, dictionary_lookup, web_search]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print(f"✅ LLM bound to {len(tools)} tools")
print(f"   Tools: {[tool.name for tool in tools]}")

✅ LLM bound to 3 tools
   Tools: ['get_weather', 'dictionary_lookup', 'web_search']


In [29]:
# Test: Does LLM decide NOT to call tools for simple queries?
response2 = llm_with_tools.invoke([HumanMessage(content="Hello! How are you?")])

print(f"Content: {response2.content}")
print(f"Tool calls: {response2.tool_calls}")

Content: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
Tool calls: []


In [30]:
sys_msg = SystemMessage(content="""You are a helpful assistant with access to tools
When asked about the weather and atmosphere condition, use the get_weather tool.
When asked meaning of words, use the dictionary_lookup tool.
when asked to search the web, use the web_search tool.
Only use the tool when necessary, answer directly for simple questions""")

def assistant(state: MessagesState) -> dict:
  """
  Assistant node - decides whether to use tools or answer directly.
  """
  messages = [sys_msg] + state["messages"]
  response = llm_with_tools.invoke(messages)
  return {"messages": [response]}

print("Assistant with node defined")

Assistant with node defined


In [31]:
# Defining the conditional routing
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
  """
  Decide next step based on the last message.

  If LLM called a tool ➡️ go to 'tools' node
  If LLM provided final answer ➡️ go to end
  """
  last_message = state["messages"][-1]
    
  # Check if LLM made tool calls
  if last_message.tool_calls:
      return "tools"
    
  # No tool calls - we're done
  return "__end__"

print("✅ Conditional routing function defined")

✅ Conditional routing function defined


In [32]:
builder =StateGraph(MessagesState)

# Add Nodes 
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools)) #Executes tool call automatically

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
  "assistant", 
  should_continue,
  {"tools": "tools", "__end__": END})
builder.add_edge("tools", "assistant") #go back to assistant after tools

# Add memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agent graph compiled with tools and memory")

✅ Agent graph compiled with tools and memory


In [33]:
# Visualize the agent graph
try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Graph structure: START → assistant → [conditional] → tools → assistant → END")

Could not display graph: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`
Graph structure: START → assistant → [conditional] → tools → assistant → END


In [34]:
# Helper function
def run_agent(user_input: str, thread_id: str = "test"):
    """
    Run the agent and display the conversation.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")
    
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            continue  # Already printed
        elif isinstance(message, AIMessage):
            if message.tool_calls:
                print(f"🤖 Agent: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 Agent: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool Result: {message.content[:100]}..." if len(message.content) > 100 else f"🔧 Tool Result: {message.content}")
    
    print(f"\n{'='*70}\n")

print("✅ Test function ready")

✅ Test function ready


In [35]:
run_agent("What is the weather condition in Lagos?")


👤 User: What is the weather condition in Lagos?

🤖 Agent: [Calling tool: get_weather]
🔧 Tool Result: Weather in Lagos, Nigeria:
Temperature: 30°C
Condition: Humid and cloudy
🤖 Agent: The weather in Lagos is currently 30°C, with humid and cloudy conditions.




In [37]:
run_agent("What is voltage?")


👤 User: What is voltage?

🤖 Agent: [Calling tool: get_weather]
🔧 Tool Result: Weather in Lagos, Nigeria:
Temperature: 30°C
Condition: Humid and cloudy
🤖 Agent: The weather in Lagos is currently 30°C, with humid and cloudy conditions.
🤖 Agent: [Calling tool: dictionary_lookup]
🔧 Tool Result: Voltage:
The electrical potential difference between two points.
🤖 Agent: Voltage is defined as the electrical potential difference between two points.
🤖 Agent: Voltage is the electrical potential difference between two points in a circuit. It is a measure of the energy per unit charge that is available to drive electric current through a conductor.




In [38]:
run_agent("Power quality disturbance in electrical system?")


👤 User: Power quality disturbance in electrical system?



C:\Users\USER\AppData\Local\Temp\ipykernel_23760\3222841153.py:10: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


🤖 Agent: [Calling tool: get_weather]
🔧 Tool Result: Weather in Lagos, Nigeria:
Temperature: 30°C
Condition: Humid and cloudy
🤖 Agent: The weather in Lagos is currently 30°C, with humid and cloudy conditions.
🤖 Agent: [Calling tool: dictionary_lookup]
🔧 Tool Result: Voltage:
The electrical potential difference between two points.
🤖 Agent: Voltage is defined as the electrical potential difference between two points.
🤖 Agent: Voltage is the electrical potential difference between two points in a circuit. It is a measure of the energy per unit charge that is available to drive electric current through a conductor.
🤖 Agent: [Calling tool: web_search]
🔧 Tool Result: No results found.
🤖 Agent: Power quality disturbances in electrical systems refer to deviations from the ideal electrical power supply that can affect the performance of electrical equipment. These disturbances can manifest in various forms, including:

1. **Voltage Sags and Swells**: Sudden decreases (sags) or increases (swells)

In [39]:
run_agent("Search for the latest AI News")


👤 User: Search for the latest AI News



C:\Users\USER\AppData\Local\Temp\ipykernel_23760\3222841153.py:10: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


🤖 Agent: [Calling tool: get_weather]
🔧 Tool Result: Weather in Lagos, Nigeria:
Temperature: 30°C
Condition: Humid and cloudy
🤖 Agent: The weather in Lagos is currently 30°C, with humid and cloudy conditions.
🤖 Agent: [Calling tool: dictionary_lookup]
🔧 Tool Result: Voltage:
The electrical potential difference between two points.
🤖 Agent: Voltage is defined as the electrical potential difference between two points.
🤖 Agent: Voltage is the electrical potential difference between two points in a circuit. It is a measure of the energy per unit charge that is available to drive electric current through a conductor.
🤖 Agent: [Calling tool: web_search]
🔧 Tool Result: No results found.
🤖 Agent: Power quality disturbances in electrical systems refer to deviations from the ideal electrical power supply that can affect the performance of electrical equipment. These disturbances can manifest in various forms, including:

1. **Voltage Sags and Swells**: Sudden decreases (sags) or increases (swells)

In [41]:
run_agent("Hi, can I know you more")


👤 User: Hi, can I know you more

🤖 Agent: [Calling tool: get_weather]
🔧 Tool Result: Weather in Lagos, Nigeria:
Temperature: 30°C
Condition: Humid and cloudy
🤖 Agent: The weather in Lagos is currently 30°C, with humid and cloudy conditions.
🤖 Agent: [Calling tool: dictionary_lookup]
🔧 Tool Result: Voltage:
The electrical potential difference between two points.
🤖 Agent: Voltage is defined as the electrical potential difference between two points.
🤖 Agent: Voltage is the electrical potential difference between two points in a circuit. It is a measure of the energy per unit charge that is available to drive electric current through a conductor.
🤖 Agent: [Calling tool: web_search]
🔧 Tool Result: No results found.
🤖 Agent: Power quality disturbances in electrical systems refer to deviations from the ideal electrical power supply that can affect the performance of electrical equipment. These disturbances can manifest in various forms, including:

1. **Voltage Sags and Swells**: Sudden decr

In [42]:
run_agent("What is the capital of france")


👤 User: What is the capital of france

🤖 Agent: [Calling tool: get_weather]
🔧 Tool Result: Weather in Lagos, Nigeria:
Temperature: 30°C
Condition: Humid and cloudy
🤖 Agent: The weather in Lagos is currently 30°C, with humid and cloudy conditions.
🤖 Agent: [Calling tool: dictionary_lookup]
🔧 Tool Result: Voltage:
The electrical potential difference between two points.
🤖 Agent: Voltage is defined as the electrical potential difference between two points.
🤖 Agent: Voltage is the electrical potential difference between two points in a circuit. It is a measure of the energy per unit charge that is available to drive electric current through a conductor.
🤖 Agent: [Calling tool: web_search]
🔧 Tool Result: No results found.
🤖 Agent: Power quality disturbances in electrical systems refer to deviations from the ideal electrical power supply that can affect the performance of electrical equipment. These disturbances can manifest in various forms, including:

1. **Voltage Sags and Swells**: Sudde

Obviously no tool was called for the capital of France